In [32]:
import cv2
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data

import scipy.stats as ss

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

from PIL import Image, ImageDraw, ImageFont
import PIL

import os

import copy
import random
import time

%matplotlib inline

In [33]:
from google.colab import drive
drive.mount('./data')

Drive already mounted at ./data; to attempt to forcibly remount, call drive.mount("./data", force_remount=True).


In [34]:
PREFIX = "data/MyDrive/ColabNotebooks/Doc_recognition/Passports/"

In [35]:
data = pd.read_csv(PREFIX + "dataset/Passports.csv")
data.head()

,Unnamed: 0,file_name,H,W,channels,department,department1,department2,department3,passport_date,...,birth_place3-bottom-right-x,birth_place3-bottom-right-y,up-left-x,up-left-y,up-right-x,up-right-y,bottom-left-x,bottom-left-y,bottom-right-x,bottom-right-y
0,0,passport0.jpg,731,522,3,СУХОЛОЖСКИМ ОВД СВЕРДЛОВСКОЙ ОБЛ.,СУХОЛОЖСКИМ ОВД СВЕРДЛОВСКОЙ ОБЛ.,NaN,NaN,17.02.2022,...,-1.0,-1.0,22.0,430.0,168.0,430.0,22.0,618.0,168.0,618.0
1,1,passport1.jpg,731,522,3,ТП УФМС РОССИИ ПО РЕСП. ТАТАРСТАН (С МЕСТОМ ДИ...,ТП УФМС РОССИИ ПО РЕСП. ТАТАРСТАН,(С МЕСТОМ ДИСЛОКАЦИИ В ПОС. ДЕРБЫШКИ),NaN,20.03.1996,...,-1.0,-1.0,22.0,430.0,168.0,430.0,22.0,618.0,168.0,618.0
2,2,passport2.jpg,731,522,3,МАЛЫЧЕВСКИМ ОМ МИЛЛЕРОВСКОГО РАЙОНА РОСТОВСКОЙ...,МАЛЫЧЕВСКИМ ОМ МИЛЛЕРОВСКОГО,РАЙОНА РОСТОВСКОЙ ОБЛ.,NaN,03.11.2003,...,-1.0,-1.0,22.0,430.0,168.0,430.0,22.0,618.0,168.0,618.0
3,3,passport3.jpg,731,522,3,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛ.,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛ.,NaN,NaN,12.04.1997,...,-1.0,-1.0,22.0,430.0,168.0,430.0,22.0,618.0,168.0,618.0
4,4,passport4.jpg,731,522,3,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСП. ДАГЕСТАН В Г. ...,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО РЕСП.,ДАГЕСТАН В Г. ХАСАВЮРТ,NaN,01.07.2014,...,-1.0,-1.0,22.0,430.0,168.0,430.0,22.0,618.0,168.0,618.0


In [36]:
if str(data.loc[0, "department1"]) != 'nan':
    print(data.loc[0, "department1"])
else:
    print("nan")

СУХОЛОЖСКИМ ОВД СВЕРДЛОВСКОЙ ОБЛ.


In [37]:
crops = {
    "file_name": [],
    "text": []
}

In [38]:
field_names = [
    "department1", "department2", "department3", "passport_date", "code", "surname1", "surname2", 
    "name", "patron", "sex", "birthday", "birth_place1", "birth_place2", "birth_place3"]

In [39]:
from tqdm.notebook import tqdm

In [40]:
suffix = ["-top-left-x", "-top-left-y", "-bottom-right-x", "-bottom-right-y"]
DATASET_PREFIX = PREFIX + "dataset/crops/"
for i in tqdm(range(data.shape[0])):
    img = Image.open(PREFIX + f"dataset/images/{data.loc[i, 'file_name']}")
    for field in field_names:
        text = data.loc[i, field]
        if str(text) == 'nan':
            continue
        coords = data.loc[i, [field + suff for suff in suffix]]
        crop = img.crop(coords)
        name = f"crop-{i}-{field}.jpg"
        crop.save(DATASET_PREFIX + name)
        crops["file_name"].append(name)
        crops["text"].append(text)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [41]:
crops_df = pd.DataFrame(crops)

In [42]:
crops_df.to_csv(PREFIX + "dataset/Crops.csv")